# 第1节 欧式期权价格
<br><br/>
* [1.1 简介](#1_1)  
* [1.2 Python 代码实现计算](#1_2)  
* [1.3 细节说明](#1_3)
    * [1.3.1 参数说明](#1_3_1)
    * [1.3.2 价格和价值](#1_3_2)
    * [1.3.3 正态分布累计概率函数$N(x)$](#1_3_3)  
    * [1.3.4 欧式期权的看跌-看涨平价关系](#1_3_4)  
    * [1.3.5 计算中使用无风险利率$r$而不是资产预期收益率$\mu$](#1_3_5)  
* [1.4 参考资料](#1_4)

## <a name="1_1"><a/>1.1 简介
<br/><br/>
&nbsp; &nbsp; &nbsp; &nbsp; 
考虑期权对应的资产价格为$S(t)$, 记为$S$，它的变化过程为几何布朗运动,
$$\frac{dS}{S} = \mu dt + \sigma dz \;\;.$$
这里假设漂移率$\mu$和波动率$\sigma$都为常数，$dz$为维纳过程（布朗运动）随机项。  

&nbsp; &nbsp; &nbsp; &nbsp; 
如果我们在时刻 $t=0$ 观察，此时资产价格为$S_0$。对于执行日期为$T$，执行价格为$K$的欧式看涨或看跌期权，我们知道根据Black-Scholes-Merton 欧式期权定价公式，它们的价格可以表示为
- 看涨期权
    $$c = S_0N(d_1)-Ke^{-rT}N(d_2)\;\;,$$
- 看跌期权
    $$p = Ke^{-rT}N(-d_2)-S_0N(-d_1)\;\;.$$
这里的$r$为无风险利率，也假设为常数。$N()$函数为标准正态分布累计概率函数，可以用`scipy.stats.norm.cdf()`计算。  
$d_1$和$d_2$为:
\begin{equation}
d_1 = \frac{\ln{\frac{S_0}{K}}+(r+\frac{1}{2}\sigma^2)T}{\sigma \sqrt{T}}, \quad 
d_2 = d_1-\sigma\sqrt{T} = \frac{\ln{\frac{S_0}{K}}+(r-\frac{1}{2}\sigma^2)T}{\sigma \sqrt{T}}\;\;.
\end{equation}

<br/><br/>
&nbsp; &nbsp; &nbsp; &nbsp; 
这里只需要用到 `math` 和`scipy.stats.norm` 两个模块，Python下自带的运算符一般比调取外面包里运算符的计算效率要高，所以这里会尽量使用Python自带的运算符。   

这里实现起来比较简单，就是把价格表达式写出Python函数就可以了。

## <a name="1_2"><a/> 1.2 Python 代码实现计算
<br/><br/>

In [3]:
import math
from scipy.stats import norm

# 常数使用"math"模块下的就好
E = math.e
PI = math.pi

# 作为例子用的一组数值
r, sigma, S_0, K, T = 0.05, 0.20, 90., 100., 3.
# 欧式期权价格函数
def european_option(r, sigma, S_0, K, T):
    """ r: 无风险利率；     sigma：资产波动率；    S_0:资产初始（当前）价格；
        K: 期权执行价格；   T：期权执行时间
    """
    
    d_1 = (math.log(S_0/K)+(r+0.5*sigma*sigma)*T)/sigma/T**0.5
    d_2 = d_1 - sigma*T**0.5
    
    call_price = S_0*norm.cdf(d_1) - K*E**(-r*T)*norm.cdf(d_2)
    put_price = K*E**(-r*T)*norm.cdf(-d_2) - S_0*norm.cdf(-d_1)
    
    return (call_price, put_price)

然后我们可以检验结果是不是满足看跌-看涨平价关系式：
\begin{equation}
c+Ke^{-rT} = p+S_0
\end{equation}
假设，$r = 0.05, \; \sigma = 0.20, \; S_0 = 90, \; K = 100, \; T = 3$，使用上面的函数我们可以计算得：

In [4]:
call_price, put_price = european_option(r, sigma, S_0, K, T)
print("欧式看涨期权价格为：{0:.5}， 欧式看跌期权价格为：{1:.5} 。".format(call_price, put_price))
print("\n看跌-看涨平价关系式为：\n")
print("   {0:.5} + {1:.5} = {2:.5} + {3:.5}\n".format(call_price, K*E**(-r*T), put_price, S_0))
print("            {0:.5} = {1:.5}\n".format(call_price+K*E**(-r*T), put_price+S_0))

欧式看涨期权价格为：14.17， 欧式看跌期权价格为：10.24 。

看跌-看涨平价关系式为：

   14.17 + 86.071 = 10.24 + 90.0

            100.24 = 100.24



## <a name="1_3"><a/> 1.3 细节说明
### <a name="1_3_1"><a/> 1.3.1 变量和参数说明  
&nbsp; &nbsp; &nbsp; &nbsp; 
在欧式期权的Black-Scholes-Merton定价公式中，假设$r, \; \sigma$在考虑时间范围内不变，$S_0$是当前的资产价格，$K$和$T$是期权中约定的执行价格和执行时间。具体而言，
- $K,\;S_0$:都默认是在同种货币下的价格。
- $T$: 时间的单位是年。这里定价公式里的$T$可以直接用总月数除十二粗略地计算。而实际上无风险投资等利率的收益是按实际天数除以一整年的有效天数计，但是股票等交易品的价格变化只在交易日发生（约252天）。可能相对合理且简单的计算方式是计算期权到期时价格期望（期权对应资产价格按调整后的无风险利率在交易日变化），再用无风险利率考虑天数调整后进行贴现。但这里的简单模型中不考虑这些细节。
- $r$：市场中的无风险利率，指进行无风险投资时资产价值增长率。实际上并不存在绝对无风险的投资，但类似买国债或在信誉好的银行存入定期存款可以近似看作无风险投资。
- $\sigma$：资产价格增长率波动率，定义应该是$\sigma^2=\mbox{Var}[\frac{S(t+\Delta t)-S(t)}{S(t)}]$ 。
    
### <a name="1_3_2"><a/> 1.3.2 价格和价值  
&nbsp; &nbsp; &nbsp; &nbsp; 
感觉这两个词很容易被混用或混淆，所以简单在这里描述一下它们的特点：
- 价格（price）：一般是指市场上我们考虑的商品的标价或买卖成交价。
- 价值（value）：我们考虑的商品的“合理”价格，不一定严格等于市场上该商品的价格。”合理“这里指我们可以通过可靠的定价方式（比如无无风险套利），通过观察市场上除该商品考虑时刻价格之外的变量，估计出该商品在该时刻应有的价格。

虽然很多时候混着用也没什么问题，但它们应该是不同的。

### <a name="1_3_3"><a/> 1.3.3 正态分布累计概率函数
&nbsp; &nbsp; &nbsp; &nbsp; 
上面期权价格表达式中的$N(x)$函数为正态分布累计概率函数，为对标准正态分布（均值为0，标准差为1）概率密度从$-\infty$到$x$的积分，即
$$N(x) = \int_{-\infty}^x \frac{1}{\sqrt{2\pi}}e^{-\frac{s^2}{2}}ds\;\;.$$  
一般具体数值都是查表或直接做数值积分，这里我们直接用`scipy`模块里的`stats.norm`类，它包含了正态分布相关的一些函数。其中`norm.cdf()`即为累计概率函数。

下面左图为正态分布概率密度函数，右图为正态分布累计概率函数。
<img src="figures/1_1.png" alt="1_1" style="width:900px;height:500px;"/>

### <a name="1_3_4"><a/> 1.3.4 欧式期权的看跌-看涨平价关系
&nbsp; &nbsp; &nbsp; &nbsp; 
对于同期限和同执行价格的欧式看涨与看跌期权，有很方便使用的等式：
$$ c+Ke^{-rT} = p+S_0\;\; .$$
我们可以通过考虑两组“等价”（同盈亏）的投资组合简单说明。考虑分别持有投资组合A和组合B的情况，其中
- 组合A：
    一份欧式看涨期权，面值为$K$执行期为$T$的无风险零息债券（面值为$K$的零息债券当前价格为$Ke^{-rT}$）。  
    当前总价值为 $c+Ke^{-rT}$。
- 组合B：
    一份欧式看跌期权，一份期权对应资产（比如股票）。  
    当前总价值为 $p+S_0$。  
    
在将来期权执行期限$T$时刻，
- 如果资产价格$S_T \geq K$。则看涨期权价值为 $S_T-K$，看跌期权价值为0。 
    - 组合A的价值为 $S_T-K+K = S_T$。
    - 组合B的价值为 $0+S_T = S_T$。
- 如果资产价格$S_T < K$。则看涨期权价值为0，看跌期权价值为 $K-S_T$。
    - 组合A的价值为 $0+K = K$。
    - 组合B的价值为 $K-S_T+S_T = K$。  

&nbsp; &nbsp; &nbsp; &nbsp; 
所以无论在未来欧式期权执行期限时资产的价格$S_T$是大于或者小于执行价格$K$，组合A和组合B的即期价值都是相等的。在这种情况下，如果两个组合在初始时刻价格不相同，则存在无风险套利机会（买入较便宜的组合，卖出较贵的组合）。所以当前组合A和组合B的价格必须相等，即
    $$ c+Ke^{-rT} = p+S_0\;\; .$$

### <a name="1_3_5"><a/> 1.3.5 计算中使用无风险利率$r$而不是资产预期收益率$\mu$
&nbsp; &nbsp; &nbsp; &nbsp; 
我们已知（假设）资产价格的变化过程为一几何布朗运动，漂移率（预期收益率）为$\mu$。但Black-Scholes-Merton 欧式期权定价公式里却没有$\mu$，而是多了参数无风险利率$r$。直接原因是期权价格公式是由Black-Scholes-Merton 偏微分方程(PDE)推导出来的，该PDE具体为：
$$\frac{\partial f}{\partial t}+rS\frac{\partial f}{\partial S}+\frac{1}\sigma\frac{\partial^2f}{{\partial S}^2} = rf\;\;.$$
加上期权价格的边界条件，可以得到开头用到的欧式期权价格的解析解。解此PDE一般用变量替换加类比热传导方程的解，或者风险中性定价。

&nbsp; &nbsp; &nbsp; &nbsp; 
这里我们不推导该PDE，但是把得到它所需要的假设列一下：
1. 欧式期权的价格只取决于对应资产的即期价格$ S(t) $和时刻$ t $;
2. 可进行Delta对冲消除不确定性。我们可以构造由期权和对应资产组成的投资组合，在任意时刻，足够小的时间段内，通过调整组合中不同资产的比例，使得投资组合的价值变化的过程在该很短的时间段内是确定的（比如描述过程的微分关系中没有布朗运动项）；
3. 无无风险套利空间。无风险（没有不确定性）的资产组合在很短时间内的收益必须等同于短时间内无风险投资（比如买国债）的收益（价值增长率为$r$）。

## <a name="1_4"><a/> 1.4 参考资料
1. 《期权、期货及其他衍生产品》，John C. Hull 著，王勇、索吾林译。